# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# Reduced Data: Grid Search + Cross-Validation

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
# %autoreload 1 #would be where you need to specify the files
# %aimport comic_recs

import pandas as pd # dataframes
import os
import pickle
import numpy as np

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import Spark methods
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import (CrossValidator, ParamGridBuilder, 
                               TrainValidationSplit)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import DataFrame
from pyspark import SparkConf
from pyspark.sql import SQLContext

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

import time
import itertools
from functools import reduce
import numpy as np

### Configure Spark

In [5]:
conf = SparkConf()

# Laptop config
# conf = (conf.setMaster('local[*]')
# #         .set('spark.executor.memory', '1G') #https://stackoverflow.com/questions/48523629/spark-pyspark-an-error-occurred-while-trying-to-connect-to-the-java-server-127
#         .set('spark.driver.memory', '7G')
#         .set('spark.driver.maxResultSize', '2G'))
# #         .set('spark.executor.memory', '1G')
# #         .set('spark.driver.memory', '10G')
# #         .set('spark.driver.maxResultSize', '5G'))

# AWS instance config
conf = (conf.setMaster('local[*]')
        .set('spark.driver.memory', '25G')
        .set('spark.driver.maxResultSize', '4G'))
#         .set('spark.executor.memory', '1G') #https://stackoverflow.com/questions/48523629/spark-pyspark-an-error-occurred-while-trying-to-connect-to-the-java-server-127
#         .set('spark.driver.memory', '10G')
#         .set('spark.driver.maxResultSize', '5G')
#         )

sc = pyspark.SparkContext().getOrCreate(conf=conf)

sql_context = SQLContext(sc)

sc.setCheckpointDir('./checkpoints')
# spark.sparkContext.setCheckpointDir("hdfs://datalake/check_point_directory/als")

## Import Data

We've previously set aside the dataset into a `json` file.

In [6]:
# We have previously created a version of the transactions table 
# and filtered it down.
sold = sql_context.read.json('raw_data/als_input_filtered_190915.json')

In [7]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

In [8]:
sold.count()

846090

### ALS Model

Let's start with  train/test split.

In [9]:
random_seed = 41916

In [10]:
# Split data into training and test set
(train, test) = sold.randomSplit([.75, .25], seed=random_seed)

In [11]:
train.count()

634611

Make sure shapes make sense.

In [12]:
print(train.count(), len(train.columns))

634611 3


In [13]:
print(test.count(), len(test.columns))

211479 3


## New Grid Search

In [14]:
# # hyper-param config
# max_iters = [20, 25, 30]
# ranks = [5, 10, 20, 25]
# reg_params = [.05, .10, .15]
# alphas = [5, 25, 40]

In [15]:
# # hyper-param config
# max_iters = [30, 40]
# ranks = [25, 30]
# reg_params = [.15, .25]
# alphas = [5]

In [16]:
# # hyper-param config
# max_iters = [30]
# ranks = [5, 10]
# reg_params = [.25, .30]
# alphas = [5]

In [17]:
# # hyper-param config
# max_iters = [40]
# ranks = [30]
# reg_params = [.25]
# alphas = [5]

In [18]:
# hyper-param config
max_iters = [20, 30, 40]
ranks = [5, 20, 40, 75, 90]
reg_params = [.01, .05, .25, .50]
alphas = [5, 25, 40]

These will be user, item and rating columns, respectively, for our dataset.
- `account_id`
- `comic_id`
- `bought`

In [19]:
model_list = cr.create_als_models_list(userCol="account_id", itemCol="comic_id"
                                    ,ratingCol="bought"
                                    ,ranks=ranks
                                    ,max_iters=max_iters
                                    ,reg_params=reg_params
                                    ,alphas=alphas
                                    ,seed=random_seed
                                   )

In [20]:
model_list

[ALS_a38b5deb7934,
 ALS_1d09ae4c4ab8,
 ALS_89fba2a23277,
 ALS_cce04348d9b1,
 ALS_3d96ac14b31c,
 ALS_02685777cc09,
 ALS_6455fc87288b,
 ALS_1539214a5498,
 ALS_2a5b04385925,
 ALS_c1e1ed2e9411,
 ALS_5a8bcfceb432,
 ALS_bb46bc889a61,
 ALS_db3e8445535e,
 ALS_b59c592b37a9,
 ALS_d63f59b8ee08,
 ALS_ae6420552114,
 ALS_af827d939681,
 ALS_0d5bda0a7713,
 ALS_1c2f927d2fa7,
 ALS_da398bd93330,
 ALS_eea1341627f7,
 ALS_2d1404b50896,
 ALS_58bdcad76150,
 ALS_07f722caad5a,
 ALS_63ba164474b7,
 ALS_fbfed798284c,
 ALS_0b391e9600ef,
 ALS_d007494608ff,
 ALS_a0ac24bf6b89,
 ALS_59fec5649f10,
 ALS_1a0cc077ba9d,
 ALS_0ac11900aaa1,
 ALS_5481dd96a657,
 ALS_de651fe4cd78,
 ALS_953178fa4c21,
 ALS_cce38a94a94d,
 ALS_bfff356a7bcc,
 ALS_80bdb93ef14b,
 ALS_e345363764b2,
 ALS_08a97ea2ba89,
 ALS_909a79d768c0,
 ALS_c2d137a5d135,
 ALS_7f4fdfb9b078,
 ALS_138cd7c42acb,
 ALS_79ef2e830947,
 ALS_1ff30e54591d,
 ALS_53f493318a13,
 ALS_12b163d0a265,
 ALS_304ffa96289d,
 ALS_1cf4e77667f2,
 ALS_a4314c909c88,
 ALS_5c3b790b60fa,
 ALS_68f7da7

Check that model list has proper number of combinations.

In [21]:
len(model_list)

180

In [22]:
print( len(model_list) == (len(max_iters)*len(ranks)*len(reg_params)*len(alphas)) )

True


### Manual Cross-Validation

## Metrics

In [23]:
ROEMs = cr.get_ROEMs(sql_context, model_list, train, test, 'account_id'
                     ,'bought')

Validation ROEM #1: 0.19869719550926318
Validation ROEM #2: 0.19433399830675785
Validation ROEM #3: 0.19657179442552847
Validation ROEM #4: 0.19727919618163878
Validation ROEM #5: 0.19135726332450673
Validation ROEM #6: 0.19600523404055623
Validation ROEM #7: 0.19613417381855794
Validation ROEM #8: 0.1883132407551012
Validation ROEM #9: 0.19068610498273622
Validation ROEM #10: 0.19615494652631793
Validation ROEM #11: 0.18812054171315873
Validation ROEM #12: 0.19001151310123257
Validation ROEM #13: 0.19825515409629393
Validation ROEM #14: 0.19304004913991987
Validation ROEM #15: 0.1965050670149716
Validation ROEM #16: 0.1970328332908752
Validation ROEM #17: 0.18878406923020186
Validation ROEM #18: 0.19591794813692
Validation ROEM #19: 0.1959248359391219
Validation ROEM #20: 0.18704623543343596
Validation ROEM #21: 0.18922641743172072
Validation ROEM #22: 0.19526198351646434
Validation ROEM #23: 0.1863824686096206
Validation ROEM #24: 0.18846383383383283
Validation ROEM #25: 0.1980254962

In [24]:
best_model = cr.get_best_model(ROEMs, model_list)

Index of smallest error: 177
Smallest error:  0.1383817704950063


In [25]:
best_model

ALS_811ff35a261b

## Extracting Parameters

In [26]:
# Extract the best_modelel parameters
print('alpha: {}'.format(best_model.getAlpha()))
print('reg param: {}'.format(best_model.getRegParam()))
print('max iter: {}'.format(best_model.getMaxIter()))
print('rank: {}'.format(best_model.getRank()))

alpha: 5.0
reg param: 0.5
max iter: 40
rank: 90


Upon inspection of the ROEM history, it appears that the best model lies along the edge of our grid, so we could potentially try other parameter values beyond. For example, the best model has a rank of 40, but we only tried rank values up to 40. We could potentially try higher values. 

In [49]:
with open('support_data/roems_20190922.pkl', 'wb') as f:
    pickle.dump(ROEMs, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

---

---

---

---

In [33]:
### Use this to reload the Grid Search results

pickle_in = open('support_data/params_errs_20190912d.pkl', 'rb')
params_errs_4 = pickle.load(pickle_in)
                         

In [27]:
# # hyper-param config
# max_iters = [40]
# ranks = [50, 60, 70,80,90,100]
# reg_params = [.50]
# alphas = [5]

In [28]:
# model_list_2 = cr.create_als_models_list(userCol="account_id", itemCol="comic_id"
#                                     ,ratingCol="bought"
#                                     ,ranks=ranks
#                                     ,max_iters=max_iters
#                                     ,reg_params=reg_params
#                                     ,alphas=alphas
#                                     ,seed=random_seed
#                                    )

In [29]:
# model_list_2

In [30]:
# ROEMs_2 = cr.get_ROEMs(sql_context, model_list_2, train, test, 'account_id'
#                      ,'bought')

#### Save the best model

In [31]:
!rm -rf models/best_model_use_20190922/

In [32]:
best_model.save('models/best_model_use_20190922')

In [33]:
# with open('support_data/best_model_20190916a.pkl', 'wb') as f:
#     pickle.dump(best_model, f)c
    
# # Example - load pickle
# # pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# # pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

# pickle_in = open('support_data/params_errs_20190912d.pkl', 'rb')
# params_errs_4 = pickle.load(pickle_in)
                         

## Cross-Validation

Perform 5-fold cross-validation

In [34]:
errors = []

In [35]:
foldlist = cr.create_spark_5_fold_set(train, seed=random_seed)

In [36]:
errors = cr.perform_cv(foldlist, best_model, sql_context, 
                       'account_id', 'bought')

In [37]:
errors

[0.15406753911315962,
 0.1579989782174029,
 0.16129332990395598,
 0.15466179803591276,
 0.15237903469248276]

In [38]:
print("CV errors range: %0.2f (+/- %0.2f)" % (np.mean(errors), 
                                              np.std(errors) * 2))

CV errors range: 0.16 (+/- 0.01)


Seems pretty stable, which helps alleviate some over-fitting concerns.

## Test the Candidate Model

Test vs our holdout set.

In [39]:
model_fitted = best_model.fit(train)

In [40]:
# get predictions on test
test_preds = model_fitted.transform(test)

# Evaluate test
test_roem = cr.calculate_ROEM(sql_context, test_preds, 'account_id', 'bought')
test_roem

0.1383823245662665

Weird that test error is lower than train...but not too far off. Let's run with it for now.

## Get Factors

#### Save the item factors for future use!

In [41]:
item_factors = model_fitted.itemFactors.toPandas()

In [42]:
item_factors.shape

(790, 2)

In [43]:
item_factors.to_pickle("support_data/item_factors_20190922.pkl")

In [44]:
pd.set_option('display.max_colwidth', -1)

In [45]:
item_factors.head()

,id,features
0,60,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.10508554428815842, 0.0, 0.0, 0.01896611973643303, 0.0, 0.021102655678987503, 0.0, 0.0, 0.12686343491077423, 0.0, 0.0, 0.0, 0.022148169577121735, 0.06328777968883514, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007707503624260426, 0.0, 0.0, 0.0, 0.0, 0.005374941509217024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05052958056330681, 0.011220408603549004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11506464332342148, 0.0, 0.0, 0.0, 0.17163017392158508, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006456415634602308, 0.0, 0.015349343419075012, 0.03107871301472187, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27092358469963074, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025583656504750252, 0.0, 0.0, 0.0, 1.0591641664505005, 0.053221795707941055, 0.0, 0.0, 0.0, 0.05408709496259689, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,80,"[0.0, 0.0, 0.7849878072738647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007554573006927967, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03375270217657089, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.032801661640405655, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007697138469666243, 0.0026825189124792814, 0.3475143313407898, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013996687484905124, 0.23373940587043762, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13523250818252563, 0.0, 0.0, 0.0, 0.0, 0.2887299656867981, 0.0, 0.0, 0.050097063183784485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03493104502558708, 0.001645353389903903, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006978376302868128, 0.1833251565694809, 0.0, 0.2944970726966858]"
2,110,"[0.0, 0.009271303191781044, 0.10891015827655792, 0.0, 0.13306932151317596, 0.05098634958267212, 0.0, 0.13732244074344635, 0.0, 0.0, 0.0, 0.028821595013141632, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07727650552988052, 0.03552515059709549, 0.0, 0.07095742225646973, 0.019442008808255196, 0.0, 0.0, 0.11037711054086685, 0.0, 0.036816131323575974, 0.0, 0.0, 0.0, 0.004124191589653492, 0.0, 0.0, 0.06366446614265442, 0.0, 0.1691473424434662, 0.0, 0.11646460741758347, 0.026325108483433723, 0.10757080465555191, 0.0, 0.0, 0.09786222130060196, 0.0, 0.0, 0.0, 0.24123691022396088, 0.0, 0.0974075123667717, 0.00572208920493722, 0.04230774939060211, 0.0, 0.0, 0.1211944967508316, 0.0, 0.12937264144420624, 0.0, 0.013567048124969006, 0.10788469761610031, 0.0, 0.14548610150814056, 0.0, 0.026104725897312164, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.049218542873859406, 0.023167787119746208, 0.0, 0.0, 0.0, 0.0, 0.1114007905125618, 0.018687862902879715, 0.24303512275218964, 0.0, 0.03269924968481064, 0.0, 0.05001208558678627, 0.0, 0.9555280208587646, 0.006476032547652721, 0.0, 0.0, 0.0, 0.02301095426082611]"
3,200,"[0.0, 0.0, 0.0850071832537651, 0.0, 0.02213653363287449, 0.14085043966770172, 0.15863579511642456, 0.004325373098254204, 0.11111370474100113, 0.0, 0.0231711957603693, 0.08182089030742645, 0.03496762365102768, 0.0, 0.0, 0.4592816233634949, 0.0, 0.0, 0.0, 0.006540149915963411, 0.0, 0.0, 0.0, 0.07584462314844131, 0.0, 0.04479466751217842, 0.0, 0.0, 0.41409534215927124, 0.43086376786231995, 0.0609794519841671, 0.0, 0.0, 0.39998140931129456, 0.0, 0.0, 0.24202348291873932, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05694151669740677, 0.011646406725049019, 0.0, 0.0, 0.0014159881975501776, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2927549183368683, 0.0, 0.01944093406200409, 0.06915950775146484, 0.0542009063065052, 0.2722710967063904, 0.0, 0.019780602306127548, 0.0896054357290268, 0.0, 0.06598655134439468, 0.0, 0.08831866085529327, 0.11044882237911224, 0.0, 0.0, 0.0, 0.04315662011504173, 0.05128350108861923, 0.0, 0.04392204061150551, 0.0, 0.0, 0.11121310293674469, 0.0, 0.0, 0.0, 0.13005110621452332, 0.003682937705889344, 0.03534753620624542, 0.0, 0.039060257375240326, 0.21271245181560516, 0.03203374892473221]"
4,240,"[0.0, 0.006236200220882893, 0.07662233710289001, 0.0, 0.0, 0.0, 0.17395563423633575, 0.016023196280002594, 1.1460156440734863, 0.0, 0.040674254298210144, 0.04191849008202553, 0.0, 0.0, 0.0, 0.02772504836320877, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

## Test Retrieving the Factors

In [46]:
unpickled_items = pd.read_pickle('support_data/item_factors_20190922.pkl')

In [47]:
unpickled_items.head()

,id,features
0,60,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.10508554428815842, 0.0, 0.0, 0.01896611973643303, 0.0, 0.021102655678987503, 0.0, 0.0, 0.12686343491077423, 0.0, 0.0, 0.0, 0.022148169577121735, 0.06328777968883514, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007707503624260426, 0.0, 0.0, 0.0, 0.0, 0.005374941509217024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05052958056330681, 0.011220408603549004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11506464332342148, 0.0, 0.0, 0.0, 0.17163017392158508, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006456415634602308, 0.0, 0.015349343419075012, 0.03107871301472187, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27092358469963074, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025583656504750252, 0.0, 0.0, 0.0, 1.0591641664505005, 0.053221795707941055, 0.0, 0.0, 0.0, 0.05408709496259689, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,80,"[0.0, 0.0, 0.7849878072738647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007554573006927967, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03375270217657089, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.032801661640405655, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007697138469666243, 0.0026825189124792814, 0.3475143313407898, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013996687484905124, 0.23373940587043762, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13523250818252563, 0.0, 0.0, 0.0, 0.0, 0.2887299656867981, 0.0, 0.0, 0.050097063183784485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03493104502558708, 0.001645353389903903, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006978376302868128, 0.1833251565694809, 0.0, 0.2944970726966858]"
2,110,"[0.0, 0.009271303191781044, 0.10891015827655792, 0.0, 0.13306932151317596, 0.05098634958267212, 0.0, 0.13732244074344635, 0.0, 0.0, 0.0, 0.028821595013141632, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07727650552988052, 0.03552515059709549, 0.0, 0.07095742225646973, 0.019442008808255196, 0.0, 0.0, 0.11037711054086685, 0.0, 0.036816131323575974, 0.0, 0.0, 0.0, 0.004124191589653492, 0.0, 0.0, 0.06366446614265442, 0.0, 0.1691473424434662, 0.0, 0.11646460741758347, 0.026325108483433723, 0.10757080465555191, 0.0, 0.0, 0.09786222130060196, 0.0, 0.0, 0.0, 0.24123691022396088, 0.0, 0.0974075123667717, 0.00572208920493722, 0.04230774939060211, 0.0, 0.0, 0.1211944967508316, 0.0, 0.12937264144420624, 0.0, 0.013567048124969006, 0.10788469761610031, 0.0, 0.14548610150814056, 0.0, 0.026104725897312164, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.049218542873859406, 0.023167787119746208, 0.0, 0.0, 0.0, 0.0, 0.1114007905125618, 0.018687862902879715, 0.24303512275218964, 0.0, 0.03269924968481064, 0.0, 0.05001208558678627, 0.0, 0.9555280208587646, 0.006476032547652721, 0.0, 0.0, 0.0, 0.02301095426082611]"
3,200,"[0.0, 0.0, 0.0850071832537651, 0.0, 0.02213653363287449, 0.14085043966770172, 0.15863579511642456, 0.004325373098254204, 0.11111370474100113, 0.0, 0.0231711957603693, 0.08182089030742645, 0.03496762365102768, 0.0, 0.0, 0.4592816233634949, 0.0, 0.0, 0.0, 0.006540149915963411, 0.0, 0.0, 0.0, 0.07584462314844131, 0.0, 0.04479466751217842, 0.0, 0.0, 0.41409534215927124, 0.43086376786231995, 0.0609794519841671, 0.0, 0.0, 0.39998140931129456, 0.0, 0.0, 0.24202348291873932, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05694151669740677, 0.011646406725049019, 0.0, 0.0, 0.0014159881975501776, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2927549183368683, 0.0, 0.01944093406200409, 0.06915950775146484, 0.0542009063065052, 0.2722710967063904, 0.0, 0.019780602306127548, 0.0896054357290268, 0.0, 0.06598655134439468, 0.0, 0.08831866085529327, 0.11044882237911224, 0.0, 0.0, 0.0, 0.04315662011504173, 0.05128350108861923, 0.0, 0.04392204061150551, 0.0, 0.0, 0.11121310293674469, 0.0, 0.0, 0.0, 0.13005110621452332, 0.003682937705889344, 0.03534753620624542, 0.0, 0.039060257375240326, 0.21271245181560516, 0.03203374892473221]"
4,240,"[0.0, 0.006236200220882893, 0.07662233710289001, 0.0, 0.0, 0.0, 0.17395563423633575, 0.016023196280002594, 1.1460156440734863, 0.0, 0.040674254298210144, 0.04191849008202553, 0.0, 0.0, 0.0, 0.02772504836320877, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

---